In [1]:
import os
import pandas as pd

def combine_csvs(input_dir='output_csvs', output_file='combined_stock_data.csv'):
    combined_df = pd.DataFrame()

    for filename in os.listdir(input_dir):
        if filename.endswith('.csv'):
            file_path = os.path.join(input_dir, filename)
            try:
                df = pd.read_csv(file_path)
                combined_df = pd.concat([combined_df, df], ignore_index=True)
                print(f"🔄 Merged: {filename} ({len(df)} rows)")
            except Exception as e:
                print(f"❌ Failed to read {filename}: {e}")

    # Optional: Clean up/standardize column order
    expected_cols = ['date', 'symbol', 'open', 'close', 'high', 'low', 'volume', 'month', 'sector']
    combined_df = combined_df[[col for col in expected_cols if col in combined_df.columns]]

    # Save the final combined file
    combined_df.to_csv(output_file, index=False)
    print(f"\n✅ Final Combined CSV Saved: {output_file} with {len(combined_df)} rows")

def main():
    combine_csvs()

if __name__ == '__main__':
    main()


🔄 Merged: ADANIENT.csv (284 rows)
🔄 Merged: ADANIPORTS.csv (284 rows)
🔄 Merged: APOLLOHOSP.csv (284 rows)
🔄 Merged: ASIANPAINT.csv (284 rows)
🔄 Merged: AXISBANK.csv (284 rows)
🔄 Merged: BAJAJ-AUTO.csv (284 rows)
🔄 Merged: BAJAJFINSV.csv (284 rows)
🔄 Merged: BAJFINANCE.csv (284 rows)
🔄 Merged: BEL.csv (284 rows)
🔄 Merged: BHARTIARTL.csv (284 rows)
🔄 Merged: BPCL.csv (284 rows)
🔄 Merged: BRITANNIA.csv (284 rows)
🔄 Merged: CIPLA.csv (284 rows)
🔄 Merged: COALINDIA.csv (284 rows)
🔄 Merged: DRREDDY.csv (284 rows)
🔄 Merged: EICHERMOT.csv (284 rows)
🔄 Merged: GRASIM.csv (284 rows)
🔄 Merged: HCLTECH.csv (284 rows)
🔄 Merged: HDFCBANK.csv (284 rows)
🔄 Merged: HDFCLIFE.csv (284 rows)
🔄 Merged: HEROMOTOCO.csv (284 rows)
🔄 Merged: HINDALCO.csv (284 rows)
🔄 Merged: HINDUNILVR.csv (284 rows)
🔄 Merged: ICICIBANK.csv (284 rows)
🔄 Merged: INDUSINDBK.csv (284 rows)
🔄 Merged: INFY.csv (284 rows)
🔄 Merged: ITC.csv (284 rows)
🔄 Merged: JSWSTEEL.csv (284 rows)
🔄 Merged: KOTAKBANK.csv (284 rows)
🔄 Merged: LT.c

In [5]:
import pandas as pd

def clean_combined_stock_data(file_path='combined_stock_data.csv', output_file='cleaned_stock_data.csv'):
    df = pd.read_csv(file_path)

    print(f"📊 Original rows: {len(df)}")

    # 1. ✅ Drop duplicates
    df.drop_duplicates(inplace=True)

    # 2. 🧼 Handle missing values (optional: drop or fill)
    missing_before = df.isnull().sum()
    df.dropna(subset=['symbol', 'date', 'open', 'close', 'high', 'low', 'volume'], inplace=True)
    missing_after = df.isnull().sum()

    # 3. 📅 Convert 'date' column to datetime format
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # 4. 🔄 Ensure numeric columns are correct data types
    num_cols = ['open', 'close', 'high', 'low', 'volume']
    for col in num_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # 5. 🧹 Remove invalid rows (e.g., negative or zero volume/prices)
    df = df[(df['volume'] > 0) & (df['close'] > 0)]

    # 6. 🧾 Standardize column names (lowercase, no spaces)
    df.columns = df.columns.str.lower()

    # 7. 🗂️ Optional: Sort by date and symbol
    df.sort_values(by=['symbol', 'date'], inplace=True)

    # 8. 💾 Save cleaned file
    df.to_csv(output_file, index=False)

    print(f"✅ Cleaned data saved to {output_file}")
    print(f"🧹 Dropped rows due to missing or invalid data: {len(df)} / {missing_before.sum()} missing cleaned")

def main():
    clean_combined_stock_data()

if __name__ == '__main__':
    main()



📊 Original rows: 14200
✅ Cleaned data saved to cleaned_stock_data.csv
🧹 Dropped rows due to missing or invalid data: 14200 / 0 missing cleaned


In [7]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import mysql.connector

def upload_to_mysql(csv_file='cleaned_stock_data.csv'):
    # 🔐 MySQL connection config
    config = {
        'host': 'localhost',
        'user': 'root',
        'password': 'root',
        'database': 'stock_data'
    }

    # Load cleaned CSV
    df = pd.read_csv(csv_file)
    df['date'] = pd.to_datetime(df['date'])

    try:
        conn = mysql.connector.connect(**config)
        cursor = conn.cursor()

        insert_query = """
            INSERT INTO stocks (date, symbol, open, close, high, low, volume, month, sector)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        # Insert row by row (batch insert also possible)
        for _, row in df.iterrows():
            cursor.execute(insert_query, tuple(row))

        conn.commit()
        print(f"✅ Uploaded {len(df)} rows to MySQL database 'stock_data'.")
        
    except mysql.connector.Error as err:
        print(f"❌ MySQL error: {err}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("🔒 MySQL connection closed.")

if __name__ == '__main__':
    upload_to_mysql()



✅ Uploaded 14200 rows to MySQL database 'stock_data'.
🔒 MySQL connection closed.


In [10]:
import mysql.connector
import pandas as pd

def verify_mysql_stock_data():
    config = {
        'host': 'localhost',
        'user': 'root',
        'password': 'root',
        'database': 'stock_data'
    }

    try:
        conn = mysql.connector.connect(**config)
        cursor = conn.cursor()

        # 1. Total records
        cursor.execute("SELECT COUNT(*) FROM stocks;")
        total_rows = cursor.fetchone()[0]

        # 2. Unique symbols
        cursor.execute("SELECT COUNT(DISTINCT symbol) FROM stocks;")
        unique_symbols = cursor.fetchone()[0]

        # 3. Date range
        cursor.execute("SELECT MIN(date), MAX(date) FROM stocks;")
        min_date, max_date = cursor.fetchone()

        # 4. Sample data
        sample_df = pd.read_sql("SELECT * FROM stocks ORDER BY date DESC LIMIT 5;", conn)

        print(f"✅ Total Rows in MySQL: {total_rows}")
        print(f"📈 Unique Symbols: {unique_symbols}")
        print(f"🗓️ Date Range: {min_date.date()} to {max_date.date()}")
        print("\n🔍 Sample Data:")
        print(sample_df)

    except mysql.connector.Error as err:
        print(f"❌ MySQL error: {err}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("🔒 Connection closed.")

if __name__ == '__main__':
    verify_mysql_stock_data()


✅ Total Rows in MySQL: 14200
📈 Unique Symbols: 50
🗓️ Date Range: 2023-10-03 to 2024-11-22

🔍 Sample Data:
      id                date      symbol     open    close     high      low  \
0  14200 2024-11-22 05:30:00       WIPRO   561.95   571.65   573.60   557.90   
1    284 2024-11-22 05:30:00    ADANIENT  2101.00  2228.00  2289.70  2025.00   
2   1420 2024-11-22 05:30:00    AXISBANK  1136.65  1142.40  1147.90  1127.55   
3   1136 2024-11-22 05:30:00  ASIANPAINT  2430.00  2472.20  2493.95  2422.95   
4    568 2024-11-22 05:30:00  ADANIPORTS  1072.60  1136.75  1155.90  1054.00   

     volume    month             sector  
0   7366616  2024-11                 IT  
1  20939196  2024-11       Conglomerate  
2  16687505  2024-11         Financials  
3   3805854  2024-11  Consumer Durables  
4  27086168  2024-11     Transportation  
🔒 Connection closed.


C:\Users\bhuva\AppData\Local\Temp\ipykernel_19220\3678946997.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sample_df = pd.read_sql("SELECT * FROM stocks ORDER BY date DESC LIMIT 5;", conn)


In [1]:
import pandas as pd

# Define sector mapping data
data = {
    'symbol': [
        'ADANIENT', 'ADANIPORTS', 'APOLLOHOSP', 'ASIANPAINT', 'AXISBANK', 'BAJAJ-AUTO',
        'BAJAJFINSV', 'BAJFINANCE', 'BEL', 'BHARTIARTL', 'BPCL', 'BRITANNIA', 'CIPLA',
        'COALINDIA', 'DRREDDY', 'EICHERMOT', 'GRASIM', 'HCLTECH', 'HDFCBANK', 'HDFCLIFE',
        'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'ICICIBANK', 'INDUSINDBK', 'INFY', 'ITC',
        'JSWSTEEL', 'KOTAKBANK', 'LT', 'M&M', 'MARUTI', 'NESTLEIND', 'NTPC', 'ONGC',
        'POWERGRID', 'RELIANCE', 'SBILIFE', 'SBIN', 'SHRIRAMFIN', 'SUNPHARMA', 'TATACONSUM',
        'TATAMOTORS', 'TATASTEEL', 'TCS', 'TECHM', 'TRENT', 'ULTRACEMCO', 'WIPRO'
    ],
    'sector': [
        'Conglomerate', 'Infrastructure', 'Pharmaceuticals', 'Consumer Goods', 'Banking', 'Automobile',
        'Finance', 'Finance', 'Defense', 'Telecom', 'Energy', 'Consumer Goods', 'Pharmaceuticals',
        'Energy', 'Pharmaceuticals', 'Automobile', 'Cement', 'Technology', 'Banking', 'Finance',
        'Automobile', 'Metals', 'Consumer Goods', 'Banking', 'Banking', 'Technology', 'Consumer Goods',
        'Metals', 'Banking', 'Infrastructure', 'Automobile', 'Automobile', 'Consumer Goods', 'Utilities',
        'Energy', 'Utilities', 'Energy', 'Finance', 'Banking', 'Finance', 'Pharmaceuticals', 'Consumer Goods',
        'Automobile', 'Metals', 'Technology', 'Technology', 'Retail', 'Cement', 'Technology'
    ]
}

# Create DataFrame
sector_df = pd.DataFrame(data)

# Save to CSV
sector_df.to_csv('sector_mapping.csv', index=False)

# Confirm saved
print("✅ sector_mapping.csv saved successfully!")



✅ sector_mapping.csv saved successfully!


In [2]:
sector_df.to_csv('sector_mapping.csv', index=False)

In [4]:
print("cleaned_stock_data.csv:", pd.read_csv("cleaned_stock_data.csv").columns.tolist())
print("sector_mapping.csv:", pd.read_csv("sector_mapping.csv").columns.tolist())

cleaned_stock_data.csv: ['date', 'symbol', 'open', 'close', 'high', 'low', 'volume', 'month', 'sector']
sector_mapping.csv: ['symbol', 'sector']
